### Поиск организаций, имеющих лицензии
Реестр МО РФ получить с портала НСИ, извлечь из архива и поместитьв целевую папку as is;   
Выгрузку лицензий поместить as is, название файла должно начинаться со слова "лицензии" ("Лицензии"), формат .xlsx, лист "Лист 1";       
Данные по силовикам: as is, название файла должно начинаться со слова "силовики" ("Силовики"), формат .xlsx, лист "Лицензии+ФРМО";   
Справочник регионов тут: `C:/work/comparison_error_successful/comparison/reg_ref.csv`

In [73]:
%%time

import pandas as pd
import numpy as np
import os
import sys
from datetime import date

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов
dir_path = "C:/work/licenses"
ref_reg_path = "C:/work/comparison_error_successful/comparison/reg_ref.csv"

for file_name in os.listdir(dir_path):
    if file_name.split("_")[0] == "1.2.643.5.1.13.13.11.1461":
        ref_path = dir_path + "/" + file_name
    elif file_name.split(" ")[0].split(".")[0].lower() == "силовики":
        army_path = dir_path + "/" + file_name
    elif file_name.split("_")[0].split(".")[0].lower() == "лицензии":
        lic_path = dir_path + "/" + file_name

df_ref = pd.read_excel(ref_path, sheet_name="Справочник", index_col=None, dtype="str")
df_lic = pd.read_excel(lic_path, sheet_name="Лист 1", index_col=None, dtype="str")
df_lic = df_lic.drop(
    index=range(0, df_lic[df_lic["Unnamed: 1"] == "Дата"].index[0])
).reset_index(drop=True)
df_lic.columns = df_lic.iloc[0]
df_lic = df_lic.drop(index=0).reset_index(drop=True)
df_army = pd.read_excel(
    army_path, sheet_name="Лицензии+ФРМО", index_col=None, dtype="str"
)

df_army_s = df_army[["№", "Дополнительный признак для фильтрации"]].rename(columns = {"Дополнительный признак для фильтрации": "special_info"})
df_lic_s = df_lic[["№", "ИНН", "ОГРН"]]
med_name_list = []
for name in df_ref["medicalSubjectName"].unique():
    if "едицинская" in name:
        med_name_list.append(name)
df_ref_s = df_ref[
    (df_ref["medicalSubjectName"].isin(med_name_list)) & (df_ref["deleteDate"].isnull())
][
    [
        "oid",
        "nameFull",
        "nameShort",
        "inn",
        "ogrn",
        "regionName",
        "moDeptName",
        "parentId",
        "organizationType",
    ]
]

df_lic_inn = df_lic_s.merge(df_ref_s, left_on="ИНН", right_on="inn", how="left")
df_lic_inn["origin"] = "ИНН"
df_lic_ogrn = df_lic_s.merge(df_ref_s, left_on="ОГРН", right_on="ogrn", how="left")
df_lic_ogrn["origin"] = "ОГРН"
df_lic_result = (
    pd.concat([df_lic_inn, df_lic_ogrn]).sort_values(by="№").reset_index(drop=True)
)
df_lic_result["oid"] = df_lic_result["oid"].fillna("не найден")

for index, row in df_lic_result.iterrows():
    if pd.isnull(row["oid"]):
        df_lic_result.loc[index, "origin"] = ""
for lic_num in df_lic_result["№"].unique():
    df_lic_0 = df_lic_result[df_lic_result["№"] == lic_num]
    for oid_num in df_lic_0["oid"].unique():
        df_lic_1 = df_lic_0[df_lic_0["oid"] == oid_num]
        if df_lic_1.shape[0] == 2:
            df_lic_result.loc[
                df_lic_1.index[0] : df_lic_1.index[-1], "origin"
            ] = "ИНН и ОГРН"
df_lic_result.drop_duplicates(inplace=True)
df_lic_result_a = df_lic_result.copy()
df_lic_result = df_lic_result_a.merge(df_army_s, left_on="№", right_on="№", how="left")
df_lic_result.drop(["inn", "ogrn"], axis=1, inplace=True)

# новый подход и корректный список регионов
df_lic_report = pd.DataFrame()
for l_num in df_lic_result["№"].unique().tolist():
    df_tmp = df_lic_result[df_lic_result["№"] == l_num]
    if df_tmp.shape[0] == 1:
        df_tmp["report"] = df_tmp["oid"]
    else:
        if (
            len(df_tmp["origin"].unique()) == 1
            and df_tmp["parentId"].isna().sum() == 1
            and set(df_tmp[~df_tmp["parentId"].isna()]["parentId"].unique()).issubset(df_tmp["oid"].unique())
            ):
            df_tmp_0 = df_tmp[df_tmp["parentId"].isna()]
            df_tmp_0["report"] = df_tmp_0["oid"]
            df_tmp_1 = df_tmp[~df_tmp["parentId"].isna()]
            df_tmp_1["report"] = ""
            df_tmp = pd.concat([df_tmp_0, df_tmp_1])
        else:
            df_tmp["report"] = "требуется ручная проверка"
    df_lic_report = pd.concat([df_lic_report, df_tmp])
ref_reg_path = "C:/work/comparison_error_successful/comparison/reg_ref.csv"
df_reg_ref = pd.read_csv(ref_reg_path, sep=";")
df_lic_report = df_lic_report.merge(
    df_reg_ref, left_on="regionName", right_on="region_name_file", how="left"
)
df_lic_report = df_lic_report.drop("region_name_file", axis=1)
df_lic_report = df_lic_report[['№', 'ИНН', 'ОГРН', 'oid', 'nameFull', 'nameShort', 'regionName','real_region_name',
       'moDeptName', 'parentId', 'organizationType', 'origin', 'special_info',
       'report']]

today = date.today().strftime("%Y-%m-%d")

lic_inn_dict = {}
for lic_num in df_lic_inn["№"].unique():
    df_lic_l = df_lic_inn[df_lic_inn["№"] == lic_num]
    lic_inn_dict[lic_num] = df_lic_l["oid"].tolist()
df_inn = pd.DataFrame(columns=["№", "найдено по ИНН"])
for key, value in lic_inn_dict.items():
    df_inn.loc[len(df_inn.index)] = [key, value]

lic_ogrn_dict = {}
for lic_num in df_lic_ogrn["№"].unique():
    df_lic_l = df_lic_ogrn[df_lic_ogrn["№"] == lic_num]
    lic_ogrn_dict[lic_num] = df_lic_l["oid"].tolist()
df_ogrn = pd.DataFrame(columns=["№", "найдено по ОГРН"])
for key, value in lic_ogrn_dict.items():
    df_ogrn.loc[len(df_ogrn.index)] = [key, value]
df_lic_list = df_inn.merge(df_ogrn, left_on="№", right_on="№")

path_save = dir_path + "/" + "licenses_report_" + today + ".xlsx"
writer = pd.ExcelWriter(path_save, engine="xlsxwriter")
# df_lic_result.to_excel(writer, sheet_name="лицензии+НСИ", index=False)
df_lic_report.to_excel(writer, sheet_name="лицензии", index=False)
df_lic_list.to_excel(writer, sheet_name="oid списком", index=False)
writer.close()
# df_lic_result.head(5)

CPU times: total: 25.8 s
Wall time: 1min 18s
